In [ ]:
import torch
import pickle
from control_torch import drss, set_seed, forced_response, tf2ss, c2d, perturb_matrices
import numpy as np

In [ ]:
# Compute settings
cuda_device = "cuda:0"
no_cuda = False
threads = 10
compile = False

# Configure compute
torch.set_num_threads(threads) 
use_cuda = not no_cuda and torch.cuda.is_available()
device_name  = cuda_device if use_cuda else "cpu"
device = torch.device(device_name)
device_type = 'cuda' if 'cuda' in device_name else 'cpu' # for later use in torch.autocast
torch.set_float32_matmul_precision("high")

In [ ]:
set_seed(42)
nx, nu, ny = 2, 1, 1  # Example dimensions, modify as needed
G_0 = drss(nx, nu, ny)
perturb_percentage = 50

In [ ]:
G_0

In [ ]:
# Generate a test set of 20 perturbed matrices
test_set = []
for _ in range(30):
    perturbed_matrix = perturb_matrices(*G_0, percentage=perturb_percentage, device=device)
    test_set.append(perturbed_matrix)

# Save the test set using pickle
with open('test_set_lin_1.pkl', 'wb') as f:
    pickle.dump(test_set, f)

print("Test set of perturbed matrices saved successfully.")

In [ ]:
import os

file_path = 'test_set_lin_1.pkl'
print(f"File size: {os.path.getsize(file_path)} bytes")

In [ ]:
# Load the test set from the pickle file
with open('test_set_lin_1.pkl', 'rb') as f:
    test_set = pickle.load(f)

In [ ]:
from control_torch import drss, forced_response, tf2ss, c2d, perturb_matrices, set_seed, perturb_parameters
from signals_torch import steps_sequence
import matplotlib.pyplot as plt

In [ ]:
## PLOT STEP RESPONSE TO ENSURE STABILITY
Y = torch.empty((30, 500), device=device, dtype=torch.float32)
x_i = torch.zeros((30, 2), device=device, dtype=torch.float32)
u = torch.full((500,), 10, device=device, dtype=torch.float32)
u = u.reshape(-1, 1)
u[:50] = 0
for j, system in enumerate(test_set):
    y = forced_response(*system, u, x0=x_i[j])
    if y.dim() == 2 and y.size(1) == 1:
        y = y.squeeze(1)  # Remove the second dimension if it has size 1
    Y[j] = y


In [ ]:
plt.figure(figsize=(20, 10))  # Set the figure size
for i in range(Y.shape[0]):
    plt.plot(Y[i].cpu().numpy())

plt.xlabel('Time Step')  # X-axis label
plt.title('Responses of the test systems to a step')  # Title of the plot
plt.legend()  # Show legend
plt.grid(True)  
plt.show()  # 